In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from dataclasses import dataclass
from torchtune.modules import RMSNorm

/home/yuvrajsingh/miniconda3/envs/unsloth_env/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [3]:
@dataclass
class ModelArgs:
    #Hyperparameters

    block_size = 128
    batch_size = 16
    embeddings_dims = 256
    attn_dropout = 0.1
    no_of_heads = 32 #IMP needs to be thoroughly calculated
    dropout = 0.1
    epochs = 100
    max_lr = 3e-4
    no_of_decoder_layers = 32 #IMP needs to be thoroughly calculated
    weight_decay_optim = 0.1
    beta_1 = 0.9
    beta_2 = 0.95
    clip = 1.0
    device = 'cpu'
    no_kv_heads = 2

In [4]:
class RMENorm(nn.Module):
    def __init__(
        self,
        embeddings_dims: int = ModelArgs.embeddings_dims
    ):
        self.rmsnorm_layer = RMSNorm(dim=embeddings_dims)
        
        super().__init__()
        
    def forward(x, self):
        
        x = self.rmsnorm_layer(x)
        return x
        

In [58]:
import numpy as np
class RotaryEmbeddings(nn.Module):
    def __init__(
        self,
        embeddings_dims: int = ModelArgs.embeddings_dims,
        block_size: int = ModelArgs.block_size,
        batch_size: int = ModelArgs.batch_size
    ):
        super().__init__()
        
        self.embeddings_dims = embeddings_dims
        self.block_size = block_size
        self.batch_size = batch_size
        self.theta = 0  
        self.init_matrix(self.block_size)
        
        # print("MATRXO: ", self.rotatory_matrix)
        
    def init_matrix(self, seq_len):
            self.matrix = torch.zeros((seq_len, self.embeddings_dims, self.embeddings_dims), device=ModelArgs.device, requires_grad=False)
            for pos in range(seq_len):
                for j in range(1, self.embeddings_dims // 2):
                    self.theta = 10000 ** (-2*(pos-1) / self.embeddings_dims)
                    self.matrix[pos, 2*j + 1, 2*j + 1] = np.cos((pos*self.theta))
                    self.matrix[pos, 2*j + 1, j + 1] = -np.sin((pos* self.theta))
                    self.matrix[pos, 2*j , 2*j ] = -np.cos((pos* self.theta))
                    self.matrix[pos, 2*j + 1, 2*j + 1] = np.sin((pos* self.theta))
            return self.matrix
        
    def forward(self, x):
        # B,T,C = x.shape
        print("MATRIX:",x)
        if(x > self.block_size):
            matrix = self.init_matrix(x)
            return matrix
        else:
            matrix = self.init_matrix(self.block_size)
            
            return matrix

In [59]:
rot = RotaryEmbeddings()
res = rot(128)
res.shape

MATRIX: 128


torch.Size([128, 256, 256])

In [87]:
class RotaryAttentionHead(nn.Module):
    def __init__(
        self,
        embeddings_dims: int = ModelArgs.embeddings_dims,
        no_of_heads: int = ModelArgs.no_of_heads,
        attn_dropout: int = ModelArgs.attn_dropout
    ):
        super().__init__()
        self.head_size = embeddings_dims // no_of_heads
        self.query = nn.Linear(in_features=embeddings_dims, out_features=embeddings_dims, device=ModelArgs.device, bias=False)
        self.key = nn.Linear(in_features=embeddings_dims, out_features=embeddings_dims, device=ModelArgs.device, bias=False)
        self.value = nn.Linear(in_features=embeddings_dims, out_features=embeddings_dims, device=ModelArgs.device, bias=False)
        self.rotary_matrix = RotaryEmbeddings(embeddings_dims=embeddings_dims)
        self.dropout = nn.Dropout(p = ModelArgs.attn_dropout)
        
    def forward(self,x):
        # print(x.shape)
        batch, block_size, embeddings_dims = x.shape
        query = self.query(x)
        # print(query)
        key = self.key(x)
        values = self.value(x)
        matrix = self.rotary_matrix(block_size)
        
        # print(matrix.shape)
        # print(query.shape)
        masked = torch.tril(torch.ones((block_size, block_size), device=ModelArgs.device, requires_grad=False))
        rotary_query = matrix @ query.permute(1,2,0) # (B,T, C,C) @ (B,T,C) -> (B,C,T) = (B,T,C,T)
        rotary_key = matrix @ key.permute(1,2,0)  #  (B,T, C,C  ) @ (B,T,C) -> (B,C,T) = (B,T,C,T)
        weights = rotary_query.permute(2,0,1) @ rotary_key.permute(2,0,1).transpose(-2, -1)#(B,T,C,T) @ (B,T,C,T) = (T,C,C,T)
        weights_masked = weights.masked_fill(masked == 0, float('-inf'))
        scaled_weights = weights_masked / (torch.sqrt(torch.tensor(key.shape[-1])))
        scaled_weights = F.softmax(scaled_weights, dim=-1)
        value = scaled_weights @ values
        out = self.dropout(value)
        return out

In [88]:
rh = RotaryAttentionHead()
random_data = torch.randn((ModelArgs.batch_size, ModelArgs.block_size, ModelArgs.embeddings_dims))
res = rh(random_data)
res.shape

MATRIX: 128


torch.Size([16, 128, 256])

In [90]:
class MQA(nn.Module):
    def __init__(
        self,
        embeddings_dims: int = ModelArgs.embeddings_dims,
        block_size: int = ModelArgs.block_size,
        no_of_kv_heads: int = ModelArgs.no_of_heads,
        no_of_heads: int = ModelArgs.no_of_heads
    ):
        super().__init__()
        
        self.no_of_kv_heads = no_of_kv_heads
        self.no_of_q_heads = no_of_heads // no_of_kv_heads
        self.head_size = embeddings_dims // self.no_of_q_heads
        self.rotary_matrix = RotaryEmbeddings(embeddings_dims=embeddings_dims)
        # self.query = nn.Linear(in_features=embeddings_dims, out_features=self.head_size, device=ModelArgs.device, bias=False)
        self.key = nn.Linear(in_features=embeddings_dims, out_features=embeddings_dims, device=ModelArgs.device, bias=False)
        self.value = nn.Linear(in_features=embeddings_dims, out_features=embeddings_dims, device=ModelArgs.device, bias=False)
        self.dropout = nn.Dropout(p = ModelArgs.attn_dropout)
        self.linear_layer = nn.Linear(in_features=embeddings_dims, out_features=embeddings_dims, device=ModelArgs.device, bias=False)
        
        
        
    def scaled_dot_product(self, q, k, v, block_size, matrix):
            
            masked = torch.tril(torch.ones((block_size, block_size), device=ModelArgs.device, requires_grad=False))
            # print("Before: ")
            # print(q.shape)
            # print(torch.transpose(q, dim0=-2, dim1=-1).shape)
            # print(matrix.shape)
            # print(k.shape)
            # print(torch.transpose(k, dim0=-2, dim1=-1).shape)
            # rotary_query = matrix @ torch.transpose(q, dim0=-2, dim1=-1)
            # rotary_key = matrix @ torch.transpose(k, dim0=-2, dim1=-1)
            # print("After: ")
            # print(q.shape)
            # print(matrix.shape)
            # print(k.shape)
            masked = torch.tril(torch.ones((block_size, block_size), device=ModelArgs.device, requires_grad=False))
            rotary_query = matrix @ q.permute(1,2,0) # (B,T, C,C) @ (B,T,C) -> (B,C,T) = (B,T,C,T)
            rotary_key = matrix @ k.permute(1,2,0)  #  (B,T, C,C  ) @ (B,T,C) -> (B,C,T) = (B,T,C,T)
            weights = rotary_query.permute(2,0,1) @ rotary_key.permute(2,0,1).transpose(-2, -1)#(B,T,C,T) @ (B,T,C,T) = (T,C,C,T)
            weights_masked = weights.masked_fill(masked == 0, float('-inf'))
            scaled_weights = weights_masked / (torch.sqrt(torch.tensor(k.shape[-1])))
            scaled_weights = F.softmax(scaled_weights, dim=-1)
            value = scaled_weights @ v
            out = self.dropout(value)
            return value
    
    def forward(self,x):
        # print("MQA: ", x.shape)
        batch, block_size, embeddings_dims = x.shape
        multi_query = nn.ModuleList([nn.Linear(in_features=embeddings_dims, out_features=embeddings_dims, device=ModelArgs.device, bias=False) for _ in range(self.no_of_q_heads)])
        # query = self.query(x)
        matrix = self.rotary_matrix(block_size)
            

        key = self.key(x)
        values = self.value(x)
        # rotary_query = matrix @ torch.transpose(, dim0=1, dim1=0)
        # rotary_key = matrix @ torch.transpose(key, dim0=1, dim1=0)
        # matrix = self.rotary_matrix(block_size)
        # self.mqa = nn.ModuleList([
           
        # ])
        multi_query_concat = torch.cat([self.scaled_dot_product(query(x), key, values, block_size, matrix) for query in multi_query], dim=-1)
        # linear_layer_query = self.linear_layer(multi_query_concat)
        # masked = torch.tril(torch.ones((block_size, block_size), device=ModelArgs.device, requires_grad=False))
        # rotary_query = matrix @ torch.transpose(query, dim0=1, dim1=0) # (B,T,C ) @ (B,T,C,C) -> (B,C,T)
        # rotary_key = matrix @ torch.transpose(key, dim0=1, dim1=0) # (B,T,C ) @ (B,T,C,C) -> (B,C,T)
        # print(multi_query_concat.shape)
        # print(key.shape)
        # print(linear_layer_query.shape)
        # weights = linear_layer_query @ (torch.transpose(key, dim0=-2, dim1=-1))
        # weights_masked = weights.masked_fill(masked == 0, float('-inf'))
        # scaled_weights = weights_masked / (key.shape[-1] ** -0.5)
        # scaled_weights = F.softmax(scaled_weights, dim=-1)
        # value = scaled_weights @ values
        
        linear_layer= self.linear_layer(multi_query_concat)
        out = self.dropout(linear_layer)
        return out

In [96]:
class GQA(nn.Module):
    def __init__(
        self,
        embeddings_dims: int = ModelArgs.embeddings_dims,
        block_size: int = ModelArgs.block_size,
        no_of_q_heads: int = ModelArgs.no_of_heads,
        no_of_kv_heads: int = ModelArgs.no_kv_heads
    ):
        super().__init__()
        
        # self.head_size = embeddings_dims // no_of_q_heads
        # self.query = nn.Linear(in_features=embeddings_dims, out_features=self.head_size, device=ModelArgs.device, bias=False)
        self.no_of_kv_heads = no_of_kv_heads
        self.no_of_q_heads = no_of_q_heads
        # self.key = nn.Linear(in_features=embeddings_dims, out_features=self.head_size, device=ModelArgs.device, bias=False)
        # self.value = nn.Linear(in_features=embeddings_dims, out_features=self.head_size, device=ModelArgs.device, bias=False)
        self.dropout = nn.Dropout(p = ModelArgs.attn_dropout)
        self.linear_layer = nn.Linear(in_features=embeddings_dims * self.no_of_kv_heads, out_features=embeddings_dims, device=ModelArgs.device, bias=False)
        
    # def scaled_dot_product(self, q, k, v, block_size):
            
    #         masked = torch.tril(torch.ones((block_size, block_size), device=ModelArgs.device, requires_grad=False))
    #         weights = q @ (torch.transpose(k, dim0=-2, dim1=-1))
    #         weights_masked = weights.masked_fill(masked == 0, float('-inf'))
    #         scaled_weights = weights_masked / (k.shape[-1] ** -0.5)
    #         scaled_weights = F.softmax(scaled_weights, dim=-1)
    #         value = scaled_weights @ v
    #         return value
        
        
    def forward(self,x):
        
        batch, block_size, embeddings_dims = x.shape
        mqa = nn.ModuleList([MQA(embeddings_dims=embeddings_dims, block_size=block_size) for _ in range(self.no_of_kv_heads)])
        # query = self.query(x)
        # key = self.key(x)
        # values = self.value(x)
        # matrix = self.rotary_matrix(block_size)
        grouped_query_concat = torch.cat([group(x) for group in mqa], dim=-1)
        # linear_layer_query = self.linear_layer(multi_query_concat)
        # masked = torch.tril(torch.ones((block_size, block_size), device=ModelArgs.device, requires_grad=False))
        # rotary_query = matrix @ torch.transpose(query, dim0=1, dim1=0) # (B,T,C ) @ (B,T,C,C) -> (B,C,T)
        # rotary_key = matrix @ torch.transpose(key, dim0=1, dim1=0) # (B,T,C ) @ (B,T,C,C) -> (B,C,T)
        # print(multi_query_concat.shape)
        # print(key.shape)
        # print(linear_layer_query.shape)
        # print(grouped_query_concat.shape)     
        linear_layer= self.linear_layer(grouped_query_concat)
        out = self.dropout(linear_layer)
        return out

In [98]:

random_data = torch.randn((ModelArgs.batch_size, ModelArgs.block_size, ModelArgs.embeddings_dims))
gqa = GQA()
# input_data = torch.tensor()
res = gqa(random_data)
res.shape

MATRIX: 128
MATRIX: 128


torch.Size([16, 128, 256])

In [18]:
masked = torch.tril(torch.ones((ModelArgs.block_size, ModelArgs.block_size), device=ModelArgs.device, requires_grad=False))
masked.shape

torch.Size([512, 512])

In [27]:
class KVCache:
    def __init__(
        self,
        embeddings_dims: int =  ModelArgs.embeddings_dims,
        block_size: int  = ModelArgs.block_size,
        no_of_decoder_layers: int =ModelArgs.no_of_decoder_layers
    ):
        super().__init__()
        self.head_size = embeddings_dims / no_of_decoder_layers
        self.k_cache = torch.ones((block_size, embeddings_dims, self.head_size), device=ModelArgs.device, requires_grad=False)
        self.v_cache = torch.ones((block_size, embeddings_dims, self.head_size), device=ModelArgs.device, requires_grad=False)
        self.block_size = block_size,
        self.embeddings_dims = embeddings_dims
    def update(
        self,
        k: torch.tensor,
        v: torch.tensor
    ):
        self.k_cache[:self.block_size, :self.block_size] = k
        self.v_cache = v
        
    def get(self):
        

SyntaxError: incomplete input (1877103268.py, line 23)